
# Trabalho de Análise não supervisionada

# Algoritmo K-Means

## Alunos: Walter Soares Malta e David Guimarães Rocha


In [ ]:
import plotly.express as px
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats

In [ ]:
# Definição do número de clusters a serem formados
numero_de_clusters = 3
# Maior valor do número de clusters a ser testado
maximo_de_pontos_no_grafico_elbow = 12

In [ ]:
# Define o formato de apresentação dos numeros
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# Leitura da base de dados
# amostra de 1000 para aumetnar a velocidade de teste

base = pd.read_csv('../input/us-counties-covid-19-dataset/us-counties.csv') # base cortada fornecida pelo professor

df = base[['cases','deaths']] #.sample(n=1000, random_state=1)
df = df.reset_index()
df = df.drop('index', 1)
df

In [ ]:
df.describe()

In [ ]:
# verificando a existencia de nulos
df.isnull().sum()

In [ ]:
# Coluna com a informação do centroide do cluster ao qual pertence o ponto
df['centroides'] = 1

# Tamnho do ponto a ser plotado no grafico dinamico
df['tamanho'] = 0.2

# Coluna para guardar distancia ao centroide mais proximo
df['distancia'] = 0.0

In [ ]:
# Função para calculo da distancia euclidiana
def calcula_distancia(x1,y1,x2,y2):
    return ((x2-x1)**2 +(y2-y1)**2)**(1/2)

In [ ]:
# Dataframe que guarda o historico de atualização dos centroides para geração do gráfico dinâmico
historico = pd.DataFrame([], columns = ['cases','deaths','centroides','contador'])

In [ ]:
# rotina K-Means
# Cria uma tabela para os centroides e insere pontos aleatorios colhidos da base
# Pontos é a matriz de dados
# nr_cluster é o número de clusters a ser formado
# deslocamento_minimo determina o momento de para a execução 
# caso o deslocamento do centroide na iteração for menor que este valor
# max_iter determina a parada da rotina caso o número de iterações alcançe esse valor
# v_historico é a tabela que guarda o historico

def fit_k_means(pontos,nr_clusters,deslocamento_minimo, max_iter, v_historico):
    
    v_historico = v_historico[0:0]
    maior_distancia = 20
    contador = 0
    big_value = 100000000
    
    centroides = pd.DataFrame({ 'nr' : range(1, nr_clusters + 1 ,1)})
    amostra = pontos[['cases','deaths']].sample(n=nr_clusters, random_state=1).astype(float)
    amostra = amostra.reset_index()
    amostra.drop('index',axis='columns', inplace=True)    
    centroides = pd.concat([centroides, amostra], axis=1)
    centroides['centroides'] = centroides['nr'] # Coluna inserida apenas para que se possa concatenar esse dataframe com o de dados para plotar tudo junto
    centroides['tamanho'] = 2.0 # No grafico a ser plotado esses pontos aparecerão maiores
    
    print('Iteração: ')

    # loop com o scriterios de parada
    while (maior_distancia > deslocamento_minimo) and (contador < max_iter):
        
        total = 0.0
        inercia = 0.0

        contador = contador + 1      

        centroides_old = centroides.copy() # guarda a posicao anterior dos centroides para calculo do momento de parada (maior_distancia de iteracoes)
        
        for index,row in pontos.iterrows():

            distancia_menor = big_value
            
            for index_2,row_2 in centroides.iterrows():
                
                distancia = calcula_distancia(row['cases'],row['deaths'],row_2['cases'],row_2['deaths'])
                
                if (distancia_menor == big_value) or (distancia < distancia_menor):
                    distancia_menor = distancia
                    nr_centroide = centroides.at[index_2,'nr']
                    cases = row['cases']
                    deaths = row['deaths']

            pontos.at[index,'centroides'] = nr_centroide
            pontos.at[index,'distancia'] = distancia_menor
            
            total += distancia_menor
            inercia += distancia_menor**2

        pedaco = pontos[['cases','deaths','centroides','tamanho']] 
        pedaco = pedaco.append(centroides)
        pedaco['contador'] = contador
        v_historico = v_historico.append(pedaco)    
               
        for index_3,row_3 in centroides.iterrows():

            centroides.at[index_3,'cases' ] = pontos[pontos['centroides'] == row_3['nr']]['cases'].astype(float).mean()
            centroides.at[index_3,'deaths'] = pontos[pontos['centroides'] == row_3['nr']]['deaths'].astype(float).mean()

        # maior_distancia guarda a soma total de todas as diferenças de posicao
        diferenca_centroides = centroides - centroides_old
        quadrado_diferenca=diferenca_centroides**2
        maior_distancia = np.sqrt(quadrado_diferenca['cases'] + quadrado_diferenca['deaths']).max()
        
        print(str(contador), end=' ')
        
        fit_k_means.distancia_global = total/pontos.shape[0]
        fit_k_means.inercia_global = inercia
                   
        if nr_clusters == 1:
            break
 
    return v_historico


In [ ]:
# Execução da rotina. Retorna o historico para gerar o grafico dinâmico
# A coluna contador informa a sequencia do conjunto de dados para cada imagem do grafico
historico = fit_k_means(df,numero_de_clusters,1, 100, historico)

In [ ]:
# Tabela com o histórico da formação dos clusters durante execução da rotina k-means
historico

In [ ]:
# Grafico dinâmico
max_x = historico['cases'].max()*1.05
max_y = historico['deaths'].max()*1.05
min_x = historico['cases'].max()*-0.05
min_y = historico['deaths'].max()*-0.05
px.scatter(historico, x="cases", y="deaths", color="centroides",
           animation_frame="contador", size = "tamanho", symbol="tamanho", 
           title="Formação de clusters utilizando K-Means",
           symbol_sequence = ['circle','x'],            
           range_x=[min_x,max_x], range_y=[min_y,max_y])

In [ ]:
# Distancia média dos pontos ao centroide do cluster
df.groupby('centroides').mean()['distancia'].reset_index()

In [ ]:
# Quantidade de elementos por cluster
df.groupby('centroides')['cases'].count().reset_index()

In [ ]:
# terceiro cluster contém apenas um elemento. Trata-se de um outlier
# Vamos removê-lo usando z-score
df['z_score']=stats.zscore(df['cases'])
df_o = df.loc[df['z_score'].abs()<=3]
print(f'Dimensão da base sem os outliers: {df_o.shape}')

In [ ]:
# Executaremos a rotina K-means com a base filtrada
# historico_o guarda n repetições da base com a coluna contador informando a sequencia historica

historico_o = pd.DataFrame([], columns = ['cases','deaths','centroides','contador'])
historico_o = fit_k_means(df_o,numero_de_clusters,5, 100, historico_o)

max_x = historico_o['cases'].max()*1.05
max_y = historico_o['deaths'].max()*1.05
min_x = historico_o['cases'].max()*-0.05
min_y = historico_o['deaths'].max()*-0.05

px.scatter(historico_o, x="cases", y="deaths", color="centroides",
           animation_frame="contador", size = "tamanho", symbol="tamanho", 
           title="Formação de clusters utilizando K-Means na base sem outliers",
           symbol_sequence = ['circle','x'],            
           range_x=[min_x,max_x], range_y=[min_y,max_y])

In [ ]:
# Tabela com o histórico da formação dos clusters durante execução da rotina k-means
# Base sem os outlyers
historico_o

In [ ]:
# Distancia média dos pontos ao centróide
df_o.groupby('centroides').mean()['distancia'].reset_index()

In [ ]:
# Quantidade de elementos por cluster
df_o.groupby('centroides')['cases'].count().reset_index()

In [ ]:
# Geração do gráfico elbow

# Definição do dataframe com informações para plotar o grafico elbow
elbow = pd.DataFrame([], columns = ['iteracao','distancia','inercia'])

# Execução da rotina k-means variando o nuemro de clusters
for i in range(1,maximo_de_pontos_no_grafico_elbow + 1):
    
    print('Nr de clusters: ' + str(i))
    fit_k_means(df_o,i,1, 100, historico_o)
    new_row = {'iteracao':i, 'distancia':fit_k_means.distancia_global, 'inercia':fit_k_means.inercia_global}
    #append row to the dataframe
    elbow = elbow.append(new_row, ignore_index=True)
    print(' ')
    print('---------------------------------------------')

In [ ]:
# Distancia média e inércia em cada iteração
elbow

In [ ]:
# Geração do gráfico elbow
px.line(elbow,x='iteracao', y='inercia', title="Gráfico Elbow")

### Análise dos clusters

Ao efetuar a classificação dos dados utilizando três clusters observamos que um deles continha apenas um elemento mais distante. No caso, as coordenadas do centroide do cluster coincidia com as do ponto.
Consideramos então tratar o ponto como um outlier, filtrando a base utilizando o z-score, e refazer a classificação. 7 pontos mais distantes foram removidos.
Após refazer a classificação percebemos que a distribuição dos pontos nos clusters se tornou mais homogênea.
O gráfico elbow parece indicar que 3 ou 5 clusters seriam mais adequados para classificar os dados. Acima desse valor os ganhos em termos de distancia média do centroide não são significantes.